# Realtime Crypto Trading Model

## Objectives

* Our goal is to build a model that can accuratley forecast the future price of bitcoin in order to make profitable trades.
* The model should be making inferences in realtime and should get retrained periodically.


## What to predict?

* Rather than aiming to predict a particular quote price at a set time $t$.

* Daily average price can be approximated by:
$$\bar{P}_{i}=\frac{C_{i} + H_{i} + L_{i}}{3}$$

* where Ci, Hi and Li are the close, high, and low quotes for day i, respectively.
* Let Vi be the set of k percentage variations of today’s close to the following k days average prices (often called arithmetic returns):

$$V_{i} = \bigg\{\frac{\bar{P}_{i+j}-C_{i}}{C_{i}}\bigg\}_{j=1}^k$$

* Our indicator variable is the total sum of the variations whose absolute value is above our target margin p%:

$$T_{i} = \sum_{v}\{v\; \epsilon\; V_{i}: v > p\%\;\vee\;v < -p\%\}$$

* The general idea of the variable **T** is to signal k-days periods that have several days with average daily prices clearly above the target variation.
* High positive values of **T** mean that there are several average daily prices that are p% higher than today’s close.
* Such situations are good indications of potential opportunities to issue a buy order, as we have good expectations that the prices will rise. 
* On the other hand, highly negative values of T suggest sell actions, given the prices will probably decline. Values around zero can be caused by periods with “flat” prices or by conflicting positive and negative variations that cancel each other.

We can formalise a trading strategy by implementing the following:

$$
  \text{signal} =
  \begin{cases}
  \text{sell} & \text{if $\;T\;\leq-0.1$} \\
  \text{hold}  & \text{if $\;-0.1\leq\;T\;\leq0.1$} \\
  \text{buy} & \text{if $\;T\;\geq0.1$}
  \end{cases}
$$

## Model Features

* We have defined an indicator (T) that summarizes the behavior of the price time series in the next k days. Our assumption is that there exists signal given by previous prices that predict expected future price.

### Rolling Arithmetic Mean

TODO

### Average True Range (ATR)

TODO

### Moving Average Convergence Divergence (MACD)

TODO

### Garman Volatility

TODO

In [2]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

bitcoin_aud = share.Share('BTC-AUD')
bitcoin_data = bitcoin_aud.get_historical(
    share.PERIOD_TYPE_DAY, 10,
    share.FREQUENCY_TYPE_MINUTE, 5)

df = pd.DataFrame(bitcoin_data)
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')
df.head()

,timestamp,open,high,low,close,volume
0,2020-02-05 20:45:00,14267.895508,14272.115234,14250.225586,14250.225586,0.0
1,2020-02-05 20:50:00,14235.811523,14285.805664,14235.811523,14273.457031,129941504.0
2,2020-02-05 20:55:00,14284.272461,14294.181641,14262.631836,14276.050781,84103168.0
3,2020-02-05 21:00:00,14296.575195,14296.575195,14286.344727,14286.344727,0.0
4,2020-02-05 21:05:00,14277.032227,14287.626953,14275.538086,14287.626953,124997632.0


In [3]:
# Daily Average Price
def calc_daily_avg_price(close, high, low):
    daily_avg_price = (close + high + low) / 3
    return daily_avg_price

def ewma():
    df['26 ema'] = pd.Series.ewm(df["Close"], span=26).mean()
    df['12 ema'] = pd.Series.ewm(df["Close"], span=12).mean()
    df['MACD'] = (df['12 ema'] - df['26 ema'])

In [12]:
def t_indicator(quotes, tgt_margin):
    p = quotes[['high', 'low', 'close']].mean(axis=1)
    v = (p - quotes['close'])/ quotes['close']
    return v[(v > tgt_margin) | (v < -tgt_margin)]

In [16]:
t_indicator(df, 0.005)

Series([], dtype: float64)

In [11]:
tgt_margin = 0.0005
v[(v > tgt_margin) | (v < -tgt_margin)]

0       0.000512
1      -0.000591
5      -0.000721
7      -0.000792
11     -0.000513
18      0.000826
34      0.000800
42      0.000899
43      0.000969
52     -0.000515
86     -0.000522
119    -0.000504
126    -0.000679
134     0.000708
135     0.000654
170    -0.000578
189    -0.000614
191     0.000662
204     0.001171
205     0.002527
207     0.000599
213    -0.000910
214    -0.000664
218     0.000621
220    -0.000825
223     0.001059
225    -0.000606
229    -0.000543
236     0.000622
243     0.000524
          ...   
2219   -0.000667
2220   -0.000696
2223    0.001747
2224    0.000919
2225   -0.001427
2226    0.000673
2244   -0.000550
2255    0.000610
2258    0.000617
2319    0.000524
2321    0.000692
2323    0.000568
2334   -0.000560
2356   -0.000771
2386    0.000561
2399   -0.000609
2404    0.000649
2440   -0.000571
2608    0.000526
2725    0.000540
2775    0.000597
2821    0.003674
2822    0.001623
2823   -0.000912
2824   -0.001740
2829    0.001343
2830    0.000801
2831    0.0012